In [12]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("Analysis").getOrCreate()

# Define S3 paths
s3_bucket = "s3://bls-gov-dataset/"
csv_path = s3_bucket + "bls-data/pr.data.0.Current"
json_path = s3_bucket + "datausa_population.json"

# Load CSV data into DataFrame
df_csv = spark.read.option("header", "true").option("delimiter", "\t").csv(csv_path)

# Strip leading and trailing spaces from column names
df_csv = df_csv.toDF(*[c.strip() for c in df_csv.columns])

# Apply trim to all columns
df_csv = df_csv.select([trim(col(c)).alias(c) for c in df_csv.columns])

# Cast year to int
df_csv = df_csv.withColumn("year", col("year").cast("int"))

# Load JSON data into DataFrame
df_json = spark.read.option("multiline", "true").json(json_path)


In [19]:
# Part 3.1: Calculate mean and standard deviation of annual US population (2013-2018)
df_population_filtered = df_json.filter((col("Year") >= 2013) & (col("Year") <= 2018))
df_population_stats = df_population_filtered.select(
    mean("Population").alias("Mean_Population"),
    stddev("Population").alias("StdDev_Population")
)
df_population_stats.show()

+---------------+-----------------+
|Mean_Population|StdDev_Population|
+---------------+-----------------+
|   3.17437383E8| 4257089.54152933|
+---------------+-----------------+


In [14]:
# Part 3.2: Find the best year for each series_id based on the largest annual sum of 'value' using SQL
df_csv.createOrReplaceTempView("csv_table")
df_best_year = spark.sql("""
    WITH annual_values AS (
        SELECT series_id, CAST(year AS INT) AS year, SUM(value) AS annual_value_sum
        FROM csv_table
        GROUP BY series_id, year
    ), ranked_values AS (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY series_id ORDER BY annual_value_sum DESC) AS rank
        FROM annual_values
    )
    SELECT series_id, year, ceil(round(annual_value_sum)) as max_value FROM ranked_values WHERE rank = 1
""")
df_best_year.show()

+-----------+----+---------+
|  series_id|year|max_value|
+-----------+----+---------+
|PRS30006011|2022|       21|
|PRS30006012|2022|       17|
|PRS30006013|1998|      704|
|PRS30006021|2010|       18|
|PRS30006022|2010|       13|
|PRS30006023|2014|      503|
|PRS30006031|2022|       20|
|PRS30006032|2021|       17|
|PRS30006033|1998|      701|
|PRS30006061|2022|       37|
|PRS30006062|2021|       32|
|PRS30006063|2024|      646|
|PRS30006081|2021|       24|
|PRS30006082|2021|       24|
|PRS30006083|2021|      111|
|PRS30006091|2002|       43|
|PRS30006092|2002|       45|
|PRS30006093|2013|      515|
|PRS30006101|2020|       33|
|PRS30006102|2020|       37|
+-----------+----+---------+
only showing top 20 rows


In [18]:
# Part 3.3: Report for series_id = 'PRS30006032' and period = 'Q01' with corresponding population
df_filtered_series = df_csv.filter((col("series_id") == "PRS30006032") & (col("period") == "Q01"))
df_joined = df_filtered_series.join(df_json, df_filtered_series.year == df_json.Year, "left").select(
    df_filtered_series.series_id,
    df_filtered_series.year,
    df_filtered_series.period,
    df_filtered_series.value,
    df_json.Population)
df_joined = df_joined.filter(col("Population").isNotNull())
df_joined.show()

+-----------+----+------+-----+----------+
|  series_id|year|period|value|Population|
+-----------+----+------+-----+----------+
|PRS30006032|2013|   Q01|  0.8| 311536594|
|PRS30006032|2014|   Q01| -0.1| 314107084|
|PRS30006032|2015|   Q01| -1.6| 316515021|
|PRS30006032|2016|   Q01| -1.4| 318558162|
|PRS30006032|2017|   Q01|  0.7| 321004407|
|PRS30006032|2018|   Q01|  0.4| 322903030|
|PRS30006032|2019|   Q01| -1.6| 324697795|
|PRS30006032|2020|   Q01| -6.7| 326569308|
|PRS30006032|2021|   Q01|  1.2| 329725481|
|PRS30006032|2022|   Q01|  5.6| 331097593|
+-----------+----+------+-----+----------+


In [ ]:
# Stop the Spark session
spark.stop()